<a href="https://colab.research.google.com/github/WillN202/NLU_CW/blob/main/NLU_Task2_DemoCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo Code

This code is used to demonstrate the model inferenece performance using the best saved model.

In [ ]:
!pip install torchmetrics
!pip install gensim
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchmetrics
import os.path
import pandas as pd
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
import random

TEST_DATASET_LOCATION = "/content/drive/MyDrive/AV_trial.csv"
TEST_MODEL_LOCATION = "/content/drive/MyDrive/twolinear_onernnlayer_gru_model_74_accuracy.model"
WORD2VEC_EMBEDDINGS = "/content/drive/MyDrive/demo_embeddings.model"
TRAINING_DATASET_LOCATION = "/content/drive/MyDrive/train.csv"

DEVICE = (
    "cuda:0"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(DEVICE)
torch.set_default_device(DEVICE)

cuda:0


## Model Definition

In [ ]:
class BaseGruRNN(torch.nn.Module):
  def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
    super(BaseGruRNN, self).__init__()
    self.vocab = vocab
    self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
    self.GRU_Layer = torch.nn.GRU(embedding_size, hidden_size, batch_first=True, num_layers=rnn_layers, dropout=0.1, bidirectional=is_bidirectional)

  def forward(self, x, linear_layer):
    unk_embedding = self.vocab["<UNK>"]
    #x = [re.sub(f"[{re.escape(string.punctuation)}]", "", sentence) for sentence in x]
    x = [sentence.split() for sentence in x]
    x = [[self.vocab.get(word, unk_embedding) for word in sentence ] for sentence in x]
    # TODO -> instead of padding, use pack sequence instead. Note this may break the output from the lstm (woo)
    max_len = max([len(words) for words in x])
    x = [([self.vocab["<pad>"]] * (max_len -  len(words))) + words for words in x]
    input = torch.tensor(x)

    embeddings = self.get_embedding(input)
    GRU_int_results = self.GRU_Layer(embeddings)[1]
    GRU_values = GRU_int_results[0] #Get last hidden state(s)
    result = linear_layer(GRU_values)
    return result

class OneLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      self.base = super(OneLinearLayerGruRNN, self).__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, output_size),
      )

    def forward(self, x):
      return super().forward(x, self.linear_layer)

class TwoLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      super().__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )

    def forward(self, x):
      return super().forward(x, self.linear_layer)

class ThreeLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      super(ThreeLinearLayerGruRNN, self).__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )

    def forward(self, x):
      return super().forward(x, self.linear_layer)

class FourLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      super(FourLinearLayerGruRNN, self).__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )

## Data and Model Processing

In [26]:
def generic_preprocessor(sentence):
  sentence = sentence.lower()

  return sentence

class AVDataset(Dataset):
  def __init__(self, csv_file, pre_processor=None):
    self.samples = pd.read_csv(csv_file)
    self.pre_processor = pre_processor

  def __len__(self):
      return len(self.samples)

  def __getitem__(self, index):
    sample = self.samples.iloc[index]
    sample_text = f"{sample[0]} <sep> {sample[1]}"
    return (self.pre_processor(sample_text), sample[2])

test_samples = AVDataset(TEST_DATASET_LOCATION, pre_processor=generic_preprocessor)

In [32]:
phrases = pd.read_csv(TRAINING_DATASET_LOCATION)
phrases = phrases.loc[:, "text_1":"text_2"].to_numpy().flatten().tolist()
#phrases = [re.sub(f"[{re.escape(string.punctuation)}]", "", str(phrase)).split() for phrase in phrases]
phrases = [str(phrase).split() for phrase in phrases]


def generate_word2vec_embeddings(size):
  print("Generating embeddings")
  embeddings = Word2Vec(sentences=phrases, workers=300, min_count=1, vector_size=size)
  embeddings.save(WORD2VEC_EMBEDDINGS)

  return embeddings


embeddings = Word2Vec.load(WORD2VEC_EMBEDDINGS) if os.path.exists(WORD2VEC_EMBEDDINGS) else generate_word2vec_embeddings(256)
embeddings.wv["<UNK>"] = np.random.rand(256)
embeddings.wv["<sep>"] = np.random.rand(256)
embeddings.wv["<pad>"] = np.random.rand(256)
coded_embeddings = torch.FloatTensor(embeddings.wv.vectors).to(DEVICE)
vocab = embeddings.wv.key_to_index

In [27]:
model = TwoLinearLayerGruRNN(256, 1, 256, coded_embeddings, vocab)
model.load_state_dict(torch.load(TEST_MODEL_LOCATION))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


<All keys matched successfully>

## Inference Loop and Result Reporting

In [38]:
model.eval()
torch.set_grad_enabled(False)
num_correct = 0

loader = DataLoader(test_samples, batch_size=64, shuffle=True, generator=torch.Generator(device=DEVICE))
answers = torch.Tensor().to(DEVICE)
gold_standard = torch.Tensor().to(DEVICE)

for index, value in enumerate(loader):
    data, labels = value
    labels = labels.reshape(-1,1)
    labels = labels.type(torch.FloatTensor)
    labels = labels.to(DEVICE)

    outputs = model(data)

    answers = torch.cat((answers, outputs))
    gold_standard = torch.cat((gold_standard, labels))


torch.set_grad_enabled(True)

Accuracy 0.6600000262260437


## Metrics Reporting

In [45]:
accuracy = torchmetrics.Accuracy(task="binary")
f1_score = torchmetrics.F1Score(task="binary")
precision = torchmetrics.Precision(task="binary")
recall = torchmetrics.Recall(task="binary")
mcc = torchmetrics.MatthewsCorrCoef(task="binary")
cohens_kappa = torchmetrics.CohenKappa(task="binary")

print(f"Accuracy {accuracy(answers, gold_standard):.2f}")
print(f"Macro Precision {precision(answers, gold_standard):.2f}")
print(f"Macro Recall {recall(answers, gold_standard):.2f}")
print(f"Macro F1-Score {f1_score(answers, gold_standard):.2f}")
print(f"MCC {mcc(answers, gold_standard):.2f}")
print(f"Cohens Kappa {cohens_kappa(answers, gold_standard):.2f}")

Accuracy 0.66
Macro Precision 0.64
Macro Recall 0.72
Macro F1-Score 0.68
MCC 0.32
Cohens Kappa 0.32
